## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
# import nltk
# nltk.download('stopwords')

In [3]:
with open('english') as f:
    stopwords = f.read()
#print(contents)

## DATA PREPROCESSING

In [4]:
data = pd.read_csv('data.csv')
data.shape

(20800, 5)

In [5]:
data.head

<bound method NDFrame.head of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2        

In [6]:
#checking for missing values in data set
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#replacing null values with empty string
data = data.fillna("")

In [8]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
#merging author and title columns - which are mainly going to be used here because text column can take too much time
data['content'] = data['author'] + ' ' + data['title']
data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [10]:
#stemming procedure
port_stem = PorterStemmer()

In [11]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)      #means all characters except a-zA-Z get converted to ' '
    stemmed_content = stemmed_content.lower()       #converting all words to lower case
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

data['content'] = data['content'].apply(stemming)

In [12]:
print(data['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel flynn flynn hillari clinton big woman c...
2                         consortiumnew com truth get fire
3        jessica purkiss civilian kill singl airstrik i...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman playoff schedul matchup odd n...
20797    michael la merc rachel abram maci said receiv ...
20798    alex ansari nato russia hold parallel exercis ...
20799                              david swanson keep aliv
Name: content, Length: 20800, dtype: object


In [15]:
X = data['content'].values
Y = data['label'].values

In [16]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth get fire' ...
 'michael la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep aliv']
[1 0 1 ... 0 1 1]


In [18]:
# converting text to numeric data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [19]:
print(X)

  (0, 15620)	0.28485063562728646
  (0, 13415)	0.2565896679337957
  (0, 8870)	0.3635963806326075
  (0, 8592)	0.29212514087043684
  (0, 7654)	0.24785219520671603
  (0, 6975)	0.21874169089359144
  (0, 4950)	0.233316966909351
  (0, 3780)	0.2705332480845492
  (0, 3588)	0.3598939188262559
  (0, 2949)	0.2468450128533713
  (0, 2473)	0.3676519686797209
  (0, 265)	0.27010124977708766
  (1, 16719)	0.30071745655510157
  (1, 6787)	0.1904660198296849
  (1, 5480)	0.7143299355715573
  (1, 3556)	0.26373768806048464
  (1, 2803)	0.19094574062359204
  (1, 2214)	0.3827320386859759
  (1, 1886)	0.15521974226349364
  (1, 1490)	0.2939891562094648
  (2, 15546)	0.4776725279299421
  (2, 5942)	0.39950146866471914
  (2, 5366)	0.4445630238272181
  (2, 3093)	0.5300162274340806
  (2, 2933)	0.36561461827708214
  :	:
  (20797, 14906)	0.31881045786261725
  (20797, 13065)	0.25400168580178045
  (20797, 12292)	0.27894826579153137
  (20797, 12086)	0.2535207421929807
  (20797, 10258)	0.08224225353919856
  (20797, 9548)	0.1785

## TRAIN TEST SPLITTING

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 3)

In [21]:
print(X_train)
print(Y_train)

  (0, 16914)	0.10256296995915114
  (0, 15517)	0.12764568484574915
  (0, 15230)	0.10063404152895654
  (0, 13461)	0.35518281718870576
  (0, 12745)	0.2948495985141781
  (0, 10258)	0.09913941418122417
  (0, 7853)	0.2913256772777924
  (0, 6948)	0.40815208655777224
  (0, 6815)	0.31046126564903126
  (0, 5377)	0.25082264460757747
  (0, 3610)	0.2948495985141781
  (0, 3024)	0.34751482912107723
  (0, 2179)	0.23811459501057528
  (0, 599)	0.2549158451831766
  (1, 15517)	0.12611364574912598
  (1, 13102)	0.4086842649329544
  (1, 11735)	0.22086627151433924
  (1, 9910)	0.4148413495308748
  (1, 8623)	0.2955700320742081
  (1, 7797)	0.3165527798719387
  (1, 3327)	0.31501564949573907
  (1, 2115)	0.45390994983155714
  (1, 595)	0.3214978327928851
  (2, 16740)	0.2260327046097581
  (2, 14352)	0.23092988682630727
  :	:
  (16637, 2803)	0.2789863966962158
  (16637, 1510)	0.3848401482639084
  (16637, 958)	0.4169011622371809
  (16638, 16914)	0.12455308126697211
  (16638, 15271)	0.35276641874030573
  (16638, 15230)	

## MODEL TRAINING

In [24]:
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

## MODEL EVALUATION

In [26]:
#accuracy on training data
prediction = model.predict(X_train)
score = accuracy_score(prediction, Y_train)
print(f"Accuracy on training data: {score}")

Accuracy on training data: 0.9859975961538462


In [27]:
#accuracy on testing data
prediction = model.predict(X_test)
score = accuracy_score(prediction, Y_test)
print(f"Accuracy on training data: {score}")

Accuracy on training data: 0.9754807692307692


## PREDICTION SYSTEM

In [36]:
input_data = X_test[1]

prediction = model.predict(input_data)
print(prediction)

if(prediction[0] == [0]): print("Real News ") 
else: print("Fake News ")

[0]
Real News 


In [37]:
print(Y_test[1])

0


## SAVING THE MODEL

In [ ]:
from joblib import dump, load
dump(classifier, 'model.joblib')